<a href="https://colab.research.google.com/github/chevamikado/ImageClassificationCNN/blob/master/ModelFromScratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Author: Muhammed Zahid Bozkus
## Lecture: INF003 - Deep Neural Networks

In [ ]:
## Import Libraries

import os
import time
import zipfile
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras # Tensorflow high-level api
from tensorflow.keras import layers

from keras import optimizers
from keras.optimizers import Adam
from keras.models import Model, Sequential
from keras.models import load_model
from keras.layers import Dense,Flatten, GlobalAveragePooling2D, BatchNormalization, Activation, Dropout, Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, CSVLogger, ReduceLROnPlateau
#from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications.xception import Xception
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
## Install the Kaggle API

!pip install kaggle

In [ ]:
## Mount the drive to colab notebook

from google.colab import files

In [ ]:
## Upload your Kaggle API Token

files.upload()

In [ ]:
## Before importing the dataset we want to use this code
## The Kaggle API client expects this file to be in ~/.kaggle

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

## This permissions change avoids a warning on Kaggle tool startup.

!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
## Download the dataset from Kaggle

!kaggle datasets download -d miljan/stanford-dogs-dataset-traintest

In [ ]:
## Unzip the dataset

local_zip = '/content/stanford-dogs-dataset-traintest.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/stanford-dogs')
zip_ref.close()

In [ ]:
## Paths for the train set and test set

train_data_dir = os.path.join("/content", "stanford-dogs", "cropped", "train")
test_data_dir = os.path.join("/content", "stanford-dogs", "cropped", "test")

In [ ]:
## Specify width-height for images and batch size

img_width, img_height = 128, 128
batch_size = 32

In [ ]:
## Generate batches of tensor image data with real-time data augmentation

train_datagen = ImageDataGenerator(
        rescale=1./255,             
        horizontal_flip = True,
        width_shift_range=0.1,
        height_shift_range=0.1,
        validation_split=0.5 
        ##vertical_flip = True,
        ##rotation_range=20,
        ##shear_range=0.05,           
        ##zoom_range=0.2,             
        #channel_shift_range=0.1
)

In [ ]:
## Generate bundle of augmented data. In the end we have a generator for the train, validation and test sets. 

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical',    # 2D one-hot encoded labels (batch_size x 101)
       )
validation_generator = train_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical',    # 2D one-hot encoded labels (batch_size x 101)
        subset='training')
test_generator = train_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_height, img_width),
        batch_size=1,
        class_mode='categorical',
        subset='validation')

In [ ]:
## Build CNN Architecture

model = Sequential()
model.add(BatchNormalization(input_shape=(128, 128, 3)))
model.add(Conv2D(filters=16, kernel_size=3, kernel_initializer='he_normal', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization())

model.add(Conv2D(filters=32, kernel_size=3, kernel_initializer='he_normal', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization())

model.add(Conv2D(filters=64, kernel_size=3, kernel_initializer='he_normal', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization())

model.add(Conv2D(filters=128, kernel_size=3, kernel_initializer='he_normal', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization())

model.add(Conv2D(filters=256, kernel_size=3, kernel_initializer='he_normal', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization())

model.add(Flatten())

model.add(Dense(120, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
## Convert the Keras model to dot format and save to a file

tf.keras.utils.plot_model(
    model,
    to_file="model.png",
    show_shapes=False,
    show_layer_names=False,
    rankdir="LR",
    expand_nested=False,
    dpi=96,
)

In [ ]:
## Specify callbacks and start training the model

checkpointer = ModelCheckpoint(filepath='newBestWeights.h5', 
                               verbose=1, save_best_only=True)

history = model.fit_generator(train_generator,
          steps_per_epoch = train_generator.n // train_generator.batch_size,
          validation_data = validation_generator,
          validation_steps = validation_generator.n // validation_generator.batch_size,
          epochs = 10,
          ##shuffle= True, 
          callbacks=[checkpointer],        
          verbose = 1)

In [ ]:
## Load the saved weights

model.load_weights('newBestWeights.h5')

In [ ]:
## Plot accuracy for the train set and validation set

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', "Validation"], loc='upper right')
plt.show()

In [ ]:
## Plot loss for the train set and validation set

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

In [ ]:
## Evaluate the model with the test set

loss, accuracy = model.evaluate(test_generator)

print("Test set loss: ", loss)
print("Test set accuracy: ", accuracy)

In [ ]:
## Train model for 10 more epochs

history = model.fit_generator(train_generator,
          steps_per_epoch = train_generator.n // train_generator.batch_size,
          validation_data = validation_generator,
          validation_steps = validation_generator.n // validation_generator.batch_size,
          epochs = 10,
          #shuffle= True, 
          callbacks=[checkpointer],        
          verbose = 1)

In [ ]:
## Evaluate the model again with the test set

loss, accuracy = model.evaluate(test_generator)

print("Test set loss: ", loss)
print("Test set accuracy: ", accuracy)